# Poisoning Attack in Federated Learning

Purpose: 
- What is the performance loss induced through the different poisoning attack? 
- What is the performance of the global model without poisoning?


In [ ]:
from federated_learning.utils import SHAPUtil, experiment_util
from federated_learning import ClientPlane, Configuration
from federated_learning.server import Server
from datetime import datetime